# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark - 056_neural_network.ipynb ##
- Keras with TensorFlow backend tested.
- For 4 layer fully connected network, R^2 = 0.673845863795


In [7]:
import time
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import savgol_filter
from scipy.stats import mode
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import colors
import matplotlib.dates as mdates
from datetime import datetime
from collections import Counter
from mpl_toolkits.basemap import Basemap
import statsmodels.api as sm
from sklearn import preprocessing
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
%matplotlib inline

### Load data

In [8]:
# Load listing data
listing = pd.read_csv('../../Data/data_after_predictor_selection.csv')
print(listing.columns.values)
listing.head()

['accommodates' 'bathrooms' 'bedrooms' 'review_scores_checkin'
 'review_scores_communication' 'latitude' 'longitude' 'property_type_0'
 'property_type_1' 'property_type_2' 'property_type_3' 'property_type_4'
 'property_type_5' 'room_type_0' 'room_type_1' 'room_type_2' 'bed_type_0'
 'bed_type_1' 'bed_type_2' 'bed_type_3' 'bed_type_4' 'beds'
 'review_scores_value' 'host_listing_count' 'review_scores_cleanliness'
 'review_scores_accuracy' 'minimum_nights' 'price']


,accommodates,bathrooms,bedrooms,review_scores_checkin,review_scores_communication,latitude,longitude,property_type_0,property_type_1,property_type_2,...,bed_type_2,bed_type_3,bed_type_4,beds,review_scores_value,host_listing_count,review_scores_cleanliness,review_scores_accuracy,minimum_nights,price
0,2,1.0,1.0,9.0,9.0,40.756852,-73.964754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,7.0,9.0,3,160
1,10,1.0,3.0,9.0,8.0,40.830599,-73.941014,1.0,0.0,0.0,...,0.0,0.0,0.0,3.0,8.0,2,8.0,8.0,1,105
2,2,1.0,1.0,10.0,10.0,40.692189,-73.924120,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,4,10.0,10.0,3,58
3,2,1.0,1.0,10.0,10.0,40.734751,-74.002592,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,9.0,10.0,10,185
4,2,1.0,1.0,10.0,10.0,40.745282,-73.997836,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,1,10.0,10.0,1,195


## Feature engineering: location clustering to price by KNN ##

In [9]:
#Step to convert longitude and latitude to price cluster by KNN
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor

class LocationPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
        super(LocationPreprocessor, self).__init__()
        
    def fit_transform(self, X, y):
        self.model.fit(X[["longitude", "latitude"]], y)
        return self.transform(X)
    
    def transform(self, X):
        loc = self.model.predict(X[["longitude", "latitude"]])
        Xnew = X.copy()
        Xnew["location"] = loc
        return Xnew

## Split data for training and testing data with transformation log(y)##

In [10]:
# Get x and y
x = listing
x = x.drop('price', axis=1)
y = listing['price']
print x.shape
print y.shape

(27392, 27)
(27392,)


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, np.log(y), test_size=0.3, random_state=0)

### Build Neural Network model ###

In [12]:
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [18]:
# define R square score function
def score(in_y_pred, in_y_act):
    
#     in_y_act = in_actual['y'].values
#     in_y_pred = in_predict['y'].values
    in_y_act_mean = in_y_act.mean()
    in_y_pred_mean = in_y_pred.mean()
    
    # input in_predict are an input dataframe
    RSS = np.multiply((in_y_act - in_y_pred), (in_y_act - in_y_pred)).sum()
    TSS = np.multiply((in_y_act - in_y_act_mean), (in_y_act - in_y_act_mean)).sum()
    #
    R_square = 1 - (RSS / TSS)
    
    return R_square

In [37]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=28, init='normal', activation='relu'))
    model.add(Dense(50, init='normal', activation='relu'))
    model.add(Dense(20, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [40]:
#fit neural network model

model = Pipeline([
            ("location", LocationPreprocessor()),
            ("normalize", StandardScaler()),
            ('mlp', KerasRegressor(build_fn=larger_model, nb_epoch=90, batch_size=10, verbose=0))
        ])

model.fit(x_train, y_train)


Epoch 1/100
19174/19174 [==============================] - 5s - loss: 0.8248     
Epoch 2/100
19174/19174 [==============================] - 4s - loss: 0.1586     
Epoch 3/100
19174/19174 [==============================] - 4s - loss: 0.1531     
Epoch 4/100
19174/19174 [==============================] - 4s - loss: 0.1517     
Epoch 5/100
19174/19174 [==============================] - 4s - loss: 0.1510     
Epoch 6/100
19174/19174 [==============================] - 4s - loss: 0.1500     
Epoch 7/100
19174/19174 [==============================] - 5s - loss: 0.1459     
Epoch 8/100
19174/19174 [==============================] - 5s - loss: 0.1443     
Epoch 9/100
19174/19174 [==============================] - 5s - loss: 0.1438     
Epoch 10/100
19174/19174 [==============================] - 5s - loss: 0.1428     
Epoch 11/100
19174/19174 [==============================] - 5s - loss: 0.1410     
Epoch 12/100
19174/19174 [==============================] - 5s - loss: 0.1411     
Epoch 13/100


ValueError: I/O operation on closed file

In [39]:
y_train_pred=model.predict(x_train)
y_test_pred=model.predict(x_test)
print 'Train Score:', score(np.reshape(y_train_pred,(y_train_pred.shape[0],)), y_train)
print 'Test Score:', score(np.reshape(y_test_pred,(y_test_pred.shape[0],)), y_test)

Train Score: 0.712446522943
Test Score: 0.673845863795
